In [35]:
import os
import json
import logging
import pandas as pd
from yt_dlp import YoutubeDL

In [36]:
def safe_filename(name):
    """Sanitize a string so it can be used safely as a filename."""
    return "".join(c if c.isalnum() or c in " _-" else "_" for c in str(name))

In [45]:
def download_and_rename_song(link, song_title, download_dir, preferredcodec):
    """
    Attempt to download a song from the given YouTube link using yt_dlp,
    then immediately rename the downloaded file to a safe filename based on song_title.
    Returns the new file path if successful, or None otherwise.
    """
    # yt_dlp options: use video ID as the temporary filename.
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(download_dir, '%(id)s.%(ext)s'),
        'quiet': False,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': preferredcodec,  # e.g., 'wav' or 'flac'
        }],
    }
    
    try:
        logging.info(f"Attempting download from: {link}")
        with YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(link, download=True)
            # Get the temporary filename (based on video id)
            temp_filename = ydl.prepare_filename(info)
    except Exception as e:
        logging.error(f"Download failed for link {link}: {e}")
        return None

    # The postprocessor should have created a file with the audio in the preferred codec.
    video_id = os.path.splitext(os.path.basename(temp_filename))[0]
    temp_filepath = os.path.join(download_dir, f"{video_id}.{preferredcodec}")
    
    if not os.path.exists(temp_filepath):
        logging.warning(f"Expected downloaded file not found: {temp_filepath}")
        return None

    # Create a safe filename based on the song title.
    new_name = safe_filename(song_title)
    new_filename = f"{new_name}.{preferredcodec}"
    new_filepath = os.path.join(download_dir, new_filename)
    
    # If a file with the desired name already exists, append a counter.
    counter = 1
    while os.path.exists(new_filepath):
        new_filename = f"{new_name}_{counter}.{preferredcodec}"
        new_filepath = os.path.join(download_dir, new_filename)
        counter += 1
    
    try:
        os.rename(temp_filepath, new_filepath)
        logging.info(f"Renamed downloaded file to: {new_filename}")
        return new_filepath
    except Exception as e:
        logging.error(f"Error renaming file {temp_filepath} to {new_filename}: {e}")
        return None

In [48]:
def download_all_songs(primary_dataset, secondary_dataset, download_dir, preferredcodec):
    """
    For each record in the primary dataset, try downloading the song and renaming it
    using its YouTube link. If that fails, look for a matching record (by song title)
    in the secondary dataset and try that link.
    A mapping from link to the new file path is saved at the end.
    """
    os.makedirs(download_dir, exist_ok=True)
    
    # Setup logging
    logging.basicConfig(level=logging.INFO, 
                        format='%(asctime)s - %(levelname)s - %(message)s')
    
    # Load primary dataset (CSV or XLSX)
    try:
        if secondary_dataset.lower().endswith('.csv'):
            secondary_df = pd.read_csv(primary_dataset)
            logging.info(f"Loaded primary CSV dataset with {len(primary_df)} entries.")
        else:
            secondary_df = pd.read_excel(primary_dataset)
            logging.info(f"Loaded primary Excel dataset with {len(primary_df)} entries.")
    except Exception as e:
        logging.error(f"Error reading primary dataset: {e}")
        return
    
    # Load secondary dataset
    secondary_df = None
    if os.path.exists(secondary_dataset):
        try:
            if secondary_dataset.lower().endswith('.csv'):
                secondary_df = pd.read_csv(secondary_dataset)
                logging.info(f"Loaded secondary CSV dataset with {len(secondary_df)} entries.")
            else:
                secondary_df = pd.read_excel(secondary_dataset)
                logging.info(f"Loaded secondary Excel dataset with {len(secondary_df)} entries.")
        except Exception as e:
            logging.error(f"Error reading secondary dataset: {e}")
            secondary_df = None
    else:
        logging.info("Secondary dataset not found, skipping alternative downloads.")
    
    downloaded_mapping = {}
    failed_downloads = []

    # Iterate over each song in the primary dataset.
    for idx, row in primary_df.iterrows():
        primary_link = row["YouTube Link"]
        song_title = row["Song"]
        logging.info(f"Processing song: '{song_title}' (primary link)")
        
        file_path = download_and_rename_song(primary_link, song_title, download_dir, preferredcodec)
        if file_path:
            downloaded_mapping[primary_link] = file_path
        else:
            logging.info(f"Primary download failed for '{song_title}'.")
            # Try secondary dataset if available.
            alternative_link = None
            if secondary_df is not None:
                # Look for a record with a matching song title.
                alt_rows = secondary_df[secondary_df["'SONG'"] == song_title]
                if not alt_rows.empty:
                    # Use the 'BEST URL FOUND' column from the secondary dataset.
                    alternative_link = alt_rows.iloc[0]["'BEST_URL_FOUND'"]
            
            if alternative_link:
                logging.info(f"Attempting secondary download for '{song_title}' from: {alternative_link}")
                file_path = download_and_rename_song(alternative_link, song_title, download_dir, preferredcodec)
                if file_path:
                    downloaded_mapping[alternative_link] = file_path
                else:
                    logging.error(f"Both primary and secondary downloads failed for '{song_title}'.")
                    failed_downloads.append(song_title)
            else:
                logging.error(f"No alternative link found for '{song_title}' in secondary dataset.")
                failed_downloads.append(song_title)

    # Save the mapping of links to new file paths.
    mapping_file = f"downloaded_mapping_{preferredcodec}_updated.json"
    try:
        with open(mapping_file, 'w') as fp:
            json.dump(downloaded_mapping, fp, indent=4)
        logging.info(f"Mapping saved to {mapping_file}")
    except Exception as e:
        logging.error(f"Error saving mapping file: {e}")

    if failed_downloads:
        logging.warning("The following songs failed to download:")
        for song in failed_downloads:
            logging.warning(song)
    else:
        logging.info("All downloads were successful!")

In [49]:
 # Paths to the datasets.
primary_dataset_file = '../datasets/Greek_Music_Dataset.xlsx'
# Change the secondary dataset path as needed (or point it to a file with alternate links).
secondary_dataset_file = "../datasets/Dataset.csv"
    
# Choose your download directory and preferred codec ('wav' or 'flac').
download_dir = '../downloads_wav_final'
preferredcodec = 'wav'
    
download_all_songs(primary_dataset_file, secondary_dataset_file, download_dir, preferredcodec)

2025-02-23 18:19:24,451 - INFO - Loaded primary dataset with 1400 entries.
2025-02-23 18:19:24,506 - INFO - Loaded secondary CSV dataset with 1000 entries.
2025-02-23 18:19:24,507 - INFO - Processing song: 'Pes Oti Thes' (primary link)
2025-02-23 18:19:24,507 - INFO - Attempting download from: http://www.youtube.com/watch?v=Uu1jAlnijVI


[youtube] Extracting URL: http://www.youtube.com/watch?v=Uu1jAlnijVI
[youtube] Uu1jAlnijVI: Downloading webpage
[youtube] Uu1jAlnijVI: Downloading tv client config
[youtube] Uu1jAlnijVI: Downloading player c8dbda2a
[youtube] Uu1jAlnijVI: Downloading tv player API JSON
[youtube] Uu1jAlnijVI: Downloading ios player API JSON
[youtube] Uu1jAlnijVI: Downloading m3u8 information
[info] Uu1jAlnijVI: Downloading 1 format(s): 251
[download] Destination: downloads_wav_final/Uu1jAlnijVI.webm
[download] 100% of    3.84MiB in 00:00:00 at 13.88MiB/s  
[ExtractAudio] Destination: downloads_wav_final/Uu1jAlnijVI.wav
Deleting original file downloads_wav_final/Uu1jAlnijVI.webm (pass -k to keep)


2025-02-23 18:19:27,206 - INFO - Renamed downloaded file to: Pes Oti Thes.wav
2025-02-23 18:19:27,207 - INFO - Processing song: 'Xartino tsirko' (primary link)
2025-02-23 18:19:27,207 - INFO - Attempting download from: http://www.youtube.com/watch?v=eXuxMWbYEro


[youtube] Extracting URL: http://www.youtube.com/watch?v=eXuxMWbYEro
[youtube] eXuxMWbYEro: Downloading webpage
[youtube] eXuxMWbYEro: Downloading tv client config
[youtube] eXuxMWbYEro: Downloading player c8dbda2a
[youtube] eXuxMWbYEro: Downloading tv player API JSON
[youtube] eXuxMWbYEro: Downloading ios player API JSON
[youtube] eXuxMWbYEro: Downloading m3u8 information
[info] eXuxMWbYEro: Downloading 1 format(s): 251
[download] Destination: downloads_wav_final/eXuxMWbYEro.webm
[download] 100% of    4.38MiB in 00:00:00 at 14.28MiB/s  
[ExtractAudio] Destination: downloads_wav_final/eXuxMWbYEro.wav
Deleting original file downloads_wav_final/eXuxMWbYEro.webm (pass -k to keep)


2025-02-23 18:19:30,080 - INFO - Renamed downloaded file to: Xartino tsirko.wav
2025-02-23 18:19:30,080 - INFO - Processing song: 'O,ti skotwneis einai diko sou gia panta' (primary link)
2025-02-23 18:19:30,081 - INFO - Attempting download from: http://www.youtube.com/watch?v=Mzz3f451dF8


...

2025-02-23 19:27:32,407 - INFO - Renamed downloaded file to: Me ta fwta nystagmena oi ntalikes.wav
2025-02-23 19:27:32,408 - INFO - Processing song: 'Ax Annoula tou xionia' (primary link)
2025-02-23 19:27:32,408 - INFO - Attempting download from: http://www.youtube.com/watch?v=aRFWJCXHCxw


[youtube] Extracting URL: http://www.youtube.com/watch?v=aRFWJCXHCxw
[youtube] aRFWJCXHCxw: Downloading webpage
[youtube] aRFWJCXHCxw: Downloading tv client config
[youtube] aRFWJCXHCxw: Downloading player c8dbda2a
[youtube] aRFWJCXHCxw: Downloading tv player API JSON
[youtube] aRFWJCXHCxw: Downloading ios player API JSON
[youtube] aRFWJCXHCxw: Downloading m3u8 information
[info] aRFWJCXHCxw: Downloading 1 format(s): 251
[download] Destination: downloads_wav_final/aRFWJCXHCxw.webm
[download] 100% of    3.35MiB in 00:00:00 at 6.28MiB/s   
[ExtractAudio] Destination: downloads_wav_final/aRFWJCXHCxw.wav
Deleting original file downloads_wav_final/aRFWJCXHCxw.webm (pass -k to keep)


2025-02-23 19:27:35,324 - INFO - Renamed downloaded file to: Ax Annoula tou xionia.wav
2025-02-23 19:27:35,325 - INFO - Processing song: 'Athina korh t' ouranou' (primary link)
2025-02-23 19:27:35,325 - INFO - Attempting download from: http://www.youtube.com/watch?v=ox42prAzC-U


[youtube] Extracting URL: http://www.youtube.com/watch?v=ox42prAzC-U
[youtube] ox42prAzC-U: Downloading webpage
[youtube] ox42prAzC-U: Downloading tv client config
[youtube] ox42prAzC-U: Downloading player c8dbda2a
[youtube] ox42prAzC-U: Downloading tv player API JSON
[youtube] ox42prAzC-U: Downloading ios player API JSON
[youtube] ox42prAzC-U: Downloading m3u8 information
[info] ox42prAzC-U: Downloading 1 format(s): 140
[download] Destination: downloads_wav_final/ox42prAzC-U.m4a
[download] 100% of    2.88MiB in 00:00:02 at 1.22MiB/s   
[FixupM4a] Correcting container of "downloads_wav_final/ox42prAzC-U.m4a"
[ExtractAudio] Destination: downloads_wav_final/ox42prAzC-U.wav
Deleting original file downloads_wav_final/ox42prAzC-U.m4a (pass -k to keep)


2025-02-23 19:27:39,764 - INFO - Renamed downloaded file to: Athina korh t_ ouranou.wav
2025-02-23 19:27:39,769 - INFO - Mapping saved to downloaded_mapping_wav_updated.json
2025-02-23 19:27:39,770 - WARNING - The following songs failed to download:
2025-02-23 19:27:39,770 - WARNING - O Koursaros
2025-02-23 19:27:39,770 - WARNING - As me lene trelh
2025-02-23 19:27:39,771 - WARNING - Xilia Peristeria
2025-02-23 19:27:39,771 - WARNING - Sempastian
2025-02-23 19:27:39,771 - WARNING - Ti na ta kaneis ta ftera
2025-02-23 19:27:39,772 - WARNING - Proswpikes Optasies
2025-02-23 19:27:39,772 - WARNING - Ta beggalika sou matia
2025-02-23 19:27:39,772 - WARNING - Poso se thelw
2025-02-23 19:27:39,773 - WARNING - Pinakas
2025-02-23 19:27:39,773 - WARNING - Pou sai Thanash
2025-02-23 19:27:39,773 - WARNING - O Maurogenhs
2025-02-23 19:27:39,773 - WARNING - Ta xwtika
2025-02-23 19:27:39,773 - WARNING - To koralli
2025-02-23 19:27:39,774 - WARNING - To petrelaio
2025-02-23 19:27:39,774 - WARNING - 

In [50]:
import os
import re

def delete_duplicate_files(download_dir):
    """
    In the given download directory, delete duplicate music files that have
    the same base name but with a counter appended (e.g., MySong_1.wav, MySong_2.wav).
    The canonical file is assumed to be the one without any counter (e.g., MySong.wav).
    If no unnumbered file exists, the file with the smallest counter is kept.
    """
    # List all files in the directory
    files = [f for f in os.listdir(download_dir) if os.path.isfile(os.path.join(download_dir, f))]

    pattern = re.compile(r"^(.*?)(?:_(\d+))?(\.[^.]+)$")
    
    # Group files by canonical name (base + extension)
    groups = {}
    for filename in files:
        match = pattern.match(filename)
        if match:
            base, counter, ext = match.groups()
            # Canonical key is the base name plus the extension (ignoring the counter)
            key = base + ext
            groups.setdefault(key, []).append((filename, int(counter) if counter else None))
        else:
            print(f"Skipping file with unmatched pattern: {filename}")
    
    # Process each group and delete duplicates
    for canonical, file_list in groups.items():
        # Determine the canonical file and delete the ones that contain counters
        no_counter = [f for f in file_list if f[1] is None]
        if no_counter:
            canonical_file = no_counter[0][0]
        else:
            # If none is without counter, choose the one with the lowest counter.
            canonical_file = sorted(file_list, key=lambda x: x[1])[0][0]
        
        # Delete all files in this group except for the canonical file.
        for filename, _ in file_list:
            if filename != canonical_file:
                try:
                    os.remove(os.path.join(download_dir, filename))
                    print(f"Deleted duplicate: {filename}")
                except Exception as e:
                    print(f"Error deleting {filename}: {e}")
                    
    print("Duplicate cleanup complete!")

In [51]:
download_directory = 'downloads_wav_final'
delete_duplicate_files(download_directory)

Deleted duplicate: Agaph_1.wav
Deleted duplicate: Akousa_1.wav
Deleted duplicate: Apopse_1.wav
Deleted duplicate: Apousia_1.wav
Deleted duplicate: Athina_1.wav
Deleted duplicate: Egw eimai edw_1.wav
Deleted duplicate: Fovamai_1.wav
Deleted duplicate: H agaph einai mia_1.wav
Deleted duplicate: Kshmerwmata_1.wav
Deleted duplicate: Stella_1.wav
Deleted duplicate: Tipota_1.wav
Deleted duplicate: Viktoria_1.wav
Deleted duplicate: Zalh_1.wav
Deleted duplicate: Zalizomai_1.wav
Deleted duplicate: Zeis_1.wav
Deleted duplicate: Zhleia_1.wav
Deleted duplicate: Zhleia_2.wav
Deleted duplicate: Zhleuw_1.wav
Deleted duplicate: Zhleuw_2.wav
Deleted duplicate: Zhleuw_3.wav
Duplicate cleanup complete!
